In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
pd.options.display.max_colwidth=999
train = pd.read_csv('../input/state-farm-distracted-driver-detection/driver_imgs_list.csv')
train[train['subject'] == 'p002']  
# p002의 c0 학습시 --> 각 subject에서 각각의 class에 대한 data가 적다보니
# 다른 subject의 똑같은 클래스(c0)여도 subject에만 집중하게되어 다른 값으로 학습해버린다
# classname에 집중해서 학습하도록 cross_val_score(교차검증) 필요함!
train['path'] = '../input/state-farm-distracted-driver-detection/imgs/train/' + train['classname'] + '/' + train['img']
train

In [ ]:
# 교차검증(KFold 기법 사용)
from sklearn.model_selection import StratifiedKFold
skf = StratifiedKFold(n_splits=5,random_state=42,shuffle=True)  # 5개로 나눔

from tensorflow.keras.preprocessing.image import ImageDataGenerator
idg = ImageDataGenerator()
idg_2 = ImageDataGenerator()
for i,(train_index,valid_index) in enumerate(skf.split(train,train['classname'])) :
    X_train = train.iloc[train_index]
    X_valid = train.iloc[valid_index]
    train_generator = idg.flow_from_dataframe(X_train,x_col='path',y_col='classname',
                                             target_size=(150,150),batch_size=32)
    valid_generator = idg_2.flow_from_dataframe(X_valid,x_col='path',y_col='classname',
                                             target_size=(150,150),batch_size=32)
    # 각 split마다 서로 다른 train_generator가 있어야 함!
    # idg_2 --> 이후 data augmentation시 valid_data가 휩쓸리지 않게 하기 위함!
    print(valid_index)
    print(len(valid_index))
# i : split 횟수(0~4)
# valid_index에는 20% 데이터  / train_index에는 80% 데이터